<a href="https://colab.research.google.com/github/markusMiksa1304/pubColab/blob/main/AudienceKW_Studie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Einloggen**

In [ ]:
from google.colab import auth
auth.authenticate_user()

# **ga simulationen**
https://towardsdatascience.com/how-to-query-and-calculate-google-analytics-data-in-bigquery-cab8fc4f396


Executing query with job ID: 618f0d8d-abec-4487-a084-8eaee4bd0b74
Query executing: 0.45s


ERROR:
 400 Unrecognized name: event_params at [21:10]

(job ID: 618f0d8d-abec-4487-a084-8eaee4bd0b74)

                                     -----Query Job SQL Follows-----                                      

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:with dataRange as (
   2:  select * FROM
   3:`tefgfs.103498486.ga_sessions_*`
   4:WHERE
   5:  _table_suffix BETWEEN '20200901' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
   6:)
   7:SELECT
   8:  event_name,
   9:  params.key AS event_parameter_key,
  10:  CASE
  11:    WHEN params.value.string_value IS NOT NULL THEN 'string'
  12:    WHEN params.value.int_value IS NOT NULL THEN 'int'
  13:    WHEN params.value.double_value IS NOT NULL THEN 'double'
  14:    WHEN params.value.float_value IS NOT NULL THEN 'float'
  15:END
  16:  AS event_parameter_value
  17:FROM
  18:  -- Change this to your Google Analytics App + Web export location in BigQ

In [ ]:
%%bigquery --project tefgfs 
SELECT
  -- User Type (dimension)
  CASE
    WHEN totals.newVisits = 1 THEN 'New Visitor'
  ELSE
  'Returning Visitor'
END
  AS User_Type,
  -- Count of Sessions (dimension)
  visitNumber AS Count_of_Sessions,
  -- Users (metric)
  COUNT(DISTINCT fullVisitorId) AS Users,
  -- New Users (metric)
  COUNT(DISTINCT(
      CASE
        WHEN totals.newVisits = 1 THEN fullVisitorId
      ELSE
      NULL
    END
      )) AS New_Users,
  -- % New Sessions (metric)
  COUNT(DISTINCT(
      CASE
        WHEN totals.newVisits = 1 THEN fullVisitorId
      ELSE
      NULL
    END
      )) / COUNT(DISTINCT CONCAT(fullVisitorId, CAST(visitStartTime AS STRING))) AS Percentage_New_Sessions,
  -- Number of Sessions per User (metric)
  COUNT(DISTINCT CONCAT(fullVisitorId, CAST(visitStartTime AS STRING))) / COUNT(DISTINCT fullVisitorId) AS Number_of_Sessions_per_User,
  -- Hits (metric)
  SUM((
    SELECT
      totals.hits
    FROM
      UNNEST(hits)
    GROUP BY
      1)) AS Hits
FROM
`tefgfs.103498486.ga_sessions_*`
WHERE
  _table_suffix BETWEEN '20200901' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
GROUP BY
  1,
  2
ORDER BY
  2
LIMIT
  10

,User_Type,Count_of_Sessions,Users,New_Users,Percentage_New_Sessions,Number_of_Sessions_per_User,Hits
0,New Visitor,1,3481145,3481145,0.998667,1.001335,122696727
1,Returning Visitor,2,1493011,0,0.000000,1.001248,49648047
2,Returning Visitor,3,864466,0,0.000000,1.001367,30117407
3,Returning Visitor,4,593358,0,0.000000,1.001333,20795571
4,Returning Visitor,5,442851,0,0.000000,1.001298,15561387
5,Returning Visitor,6,348719,0,0.000000,1.001388,12243646
6,Returning Visitor,7,284858,0,0.000000,1.001418,9954518
7,Returning Visitor,8,238344,0,0.000000,1.001217,8267146
8,Returning Visitor,9,204256,0,0.000000,1.001425,7105028
9,Returning Visitor,10,177637,0,0.000000,1.001385,6139582


**der überwiegende Teil der User kommt öfters auf unsere WebSite**

In [ ]:
%%bigquery --project tefgfs 
With dataRange as
(
  Select * From `tefgfs.103498486.ga_sessions_*`
  WHERE
  _table_suffix BETWEEN '20201001' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
)
, returnUsers as
(
SELECT
  -- User Type (dimension)
  CASE
    WHEN totals.newVisits = 1 THEN 'New Visitor'
  ELSE
  'Returning Visitor'
END
#CONCAT(fullVisitorId, CAST(visitStartTime AS STRING) as visit
  AS User_Type,
  -- Count of Sessions (dimension)
  visitNumber AS Count_of_Sessions,
  -- Users (metric)
  COUNT(DISTINCT fullVisitorId) AS Users,
  -- Hits (metric)
  SUM((
    SELECT
      totals.hits
    FROM
      UNNEST(hits)
    GROUP BY
      1)) AS Hits
FROM
 dataRange
GROUP BY
  1,
  2
ORDER BY
  2
#LIMIT
#  1000
)
Select Sum( Users) as numUsers, User_Type from returnUsers group by  User_Type

,numUsers,User_Type
0,1891588,Returning Visitor
1,842272,New Visitor


In [ ]:
%%bigquery --project tefgfs 

SELECT
  -- Page (dimension)
  pagePath AS Page,
  -- Avg Time On Page (metric)
  CASE
    WHEN pageviews = exits THEN 0
  ELSE
  total_time_on_page / (pageviews - exits)
END
  AS Avg_Time_On_Page
FROM (
  SELECT
    pagePath,
    COUNT(*) AS pageviews,
    SUM(
    IF
      (isExit IS NOT NULL,
        1,
        0)) AS exits,
    SUM(time_on_page) AS total_time_on_page
  FROM (
    SELECT
      fullVisitorId,
      visitStartTime,
      pagePath,
      hit_time,
      type,
      isExit,
      CASE
        WHEN isExit IS NOT NULL THEN last_interaction - hit_time
      ELSE
      next_pageview - hit_time
    END
      AS time_on_page
    FROM (
      SELECT
        fullVisitorId,
        visitStartTime,
        pagePath,
        hit_time,
        type,
        isExit,
        last_interaction,
        LEAD(hit_time) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hit_time) AS next_pageview
      FROM (
        SELECT
          fullVisitorId,
          visitStartTime,
          pagePath,
          hit_time,
          type,
          isExit,
          last_interaction
        FROM (
          SELECT
            fullVisitorId,
            visitStartTime,
            hits.page.pagePath,
            hits.type,
            hits.isExit,
            hits.time / 1000 AS hit_time,
            MAX(
            IF
              (hits.isInteraction = TRUE,
                hits.time / 1000,
                0)) OVER (PARTITION BY fullVisitorId, visitStartTime) AS last_interaction
          FROM
            
`tefgfs.103498486.ga_sessions_*`,
            UNNEST(hits) AS hits
          WHERE
            _table_suffix BETWEEN '20200901'
            AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
            AND totals.visits = 1
            AND hits.type = 'PAGE')))
    GROUP BY
      1,
      2,
      3,
      4,
      5,
      6,
      7)
  GROUP BY
    1)
ORDER BY
  2 DESC
LIMIT
  10

,Page,Avg_Time_On_Page
0,/e-shop/id-349966/,6024.520
1,/url/o2business-one/,5537.442
2,/e-shop/konfiguration/ot-GnlYSQOy9yyqGerYvbA/,4166.885
3,/chat-bootstrap/siteconfig/,4119.016
4,/e-shop/konfiguration/sr8bVd0YQVS5r4IRKF4ikA/,3536.773
5,/e-shop/konfiguration/xl1lwonsRaSzu-z9sIw3tg/,3132.655
6,/mobilfunk/angebote/iphone-deals/,2819.747
7,/e-shop/geraet-waehlen/bgJpGYyrSZ-BTSrkr0Doxg/,2723.959
8,/e-shop/geraet-waehlen/R89SgVpaRoyT1haaqAYGDQ/,2704.528
9,/e-shop/konfiguration/MEkAg-B8TpuLuybCHXJItg/,2701.553


In [ ]:
%%bigquery --project tefgfs 

SELECT
  -- Event Category (dimension)
  hits.eventInfo.eventCategory AS Event_Category,
  -- Event Action (dimension)
  hits.eventInfo.eventAction AS Event_Action,
  -- Event Label (dimension)
  hits.eventInfo.eventLabel AS Event_Label,
  -- Total Events (metric)
  COUNT(*) AS Total_Events,
  -- Unique Events (metric),
  COUNT(DISTINCT CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))) AS Unique_Events,
  -- Event Value (metric)
  SUM(hits.eventInfo.eventValue) AS Event_Value,
  -- Avg. Value (metric)
  SUM(hits.eventInfo.eventValue) / COUNT(*) AS Avg_Value,
  -- Sessions With Events (metric)
  COUNT(DISTINCT
    CASE
      WHEN hits.type = 'EVENT' THEN CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))
    ELSE
    NULL
  END
    ) AS Sessions_With_Events,
  -- Events / Session With Event (metric)
  COUNT(*) / COUNT(DISTINCT
    CASE
      WHEN hits.type = 'EVENT' THEN CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))
    ELSE
    NULL
  END
    ) AS Events_Session_With_Event
FROM
  `tefgfs.103498486.ga_sessions_*`,
  UNNEST(hits) AS hits
WHERE
  _table_suffix BETWEEN '20200901'
  AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
  AND totals.visits = 1
  AND hits.type = 'EVENT'
GROUP BY
  1,
  2,
  3
HAVING
  Event_Category IS NOT NULL
ORDER BY
  4 DESC
LIMIT
  10

,Event_Category,Event_Action,Event_Label,Total_Events,Unique_Events,Event_Value,Avg_Value,Sessions_With_Events,Events_Session_With_Event
0,EEC Promotion Links,Promotions,promoAttributeView1_gtm,29078848,5678531,NaN,NaN,5678531,5.120840
1,trbo,n/a - n/a - n/a - n/a,n/a - n/a - n/a - n/a,14813086,2241874,NaN,NaN,2241874,6.607457
2,Generic:Kontaktmodul,Kontaktmodul Button View,View,12540118,2880511,NaN,NaN,2880511,4.353435
3,Generic:trbo,impression,_Mein o2 - Inboxes > Inboxes mein o2 > Inbox M...,8523488,1064935,NaN,NaN,1064935,8.003764
4,Generic:trbo,impression,_Mein o2 - Inboxes > Inboxes mein o2 > Inboxes...,7599156,1047957,NaN,NaN,1047957,7.251401
5,trbo,apple - n/a - n/a - n/a,apple - n/a - n/a - n/a,6937919,1119361,NaN,NaN,1119361,6.198107
6,EEC Promotion Links,Promotions,promoAttributeView2_gtm,4825882,2632222,NaN,NaN,2632222,1.833387
7,EEC-Promotion-Banner,bannerView,banner_anzeigen_visible,2856034,997377,2856034.0,1.0,997377,2.863545
8,headerNaviElement,Click - V2 - Mein o2,Click - V2 - Mein O2,2850678,1585361,NaN,NaN,1585361,1.798125
9,trbo,n/a - BK - n/a - n/a,n/a - BK - n/a - n/a,2637592,275687,NaN,NaN,275687,9.567343


In [ ]:
%%bigquery --project tefgfs 

SELECT
  -- Transaction ID (dimension)
  hits.transaction.transactionId AS Transaction_ID,
  -- Transactions (metric)
  COUNT(DISTINCT hits.transaction.transactionId) AS Transactions,
  -- Revenue (metric)
  SUM(hits.transaction.transactionRevenue)/1000000 AS Revenue,
  -- Ecommerce Conversion Rate
  COUNT(DISTINCT hits.transaction.transactionId) / COUNT(DISTINCT CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))) AS Ecommerce_Conversion_Rate,
  -- Avg. Order Value
  (SUM(hits.transaction.transactionRevenue)/1000000)/COUNT(DISTINCT hits.transaction.transactionId) AS Avg_Order_Value,
  -- Per Session Value
  (SUM(hits.transaction.transactionRevenue)/1000000) / COUNT(DISTINCT CONCAT(CAST(fullVisitorId AS STRING), CAST(visitStartTime AS STRING))) AS Per_Session_Value,
  -- Shipping
  SUM(hits.transaction.transactionShipping)/1000000 AS Shipping,
  -- Tax
  SUM(hits.transaction.transactionTax)/1000000 AS Tax,
  -- Revenue Per User
  (SUM(hits.transaction.transactionRevenue)/1000000) / COUNT(DISTINCT fullVisitorId) AS Revenue_Per_User,
  -- Transactions Per User
  COUNT(DISTINCT hits.transaction.transactionId) / COUNT(DISTINCT fullVisitorId) AS Transactions_Per_User
FROM
  `tefgfs.103498486.ga_sessions_*`,
  UNNEST(hits) AS hits
WHERE
  _table_suffix BETWEEN '20200901'
  AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
  AND totals.visits = 1
GROUP BY
  1
HAVING
  hits.transaction.transactionId IS NOT NULL
ORDER BY
  1 DESC
LIMIT
  10

,Transaction_ID,Transactions,Revenue,Ecommerce_Conversion_Rate,Avg_Order_Value,Per_Session_Value,Shipping,Tax,Revenue_Per_User,Transactions_Per_User
0,o2-N-zzwUSIjzRump4cZy3SNj0A-2,1,2120.93,1.0,2120.93,2120.93,None,None,2120.93,1.0
1,o2-N-zzvmT1B4So-lYLHDcEWzSQ-2,1,2414.85,1.0,2414.85,2414.85,None,None,2414.85,1.0
2,o2-N-zztf-kInQueRW54L_kNOTw-2,1,2120.93,1.0,2120.93,2120.93,None,None,2120.93,1.0
3,o2-N-zzXaD-J_RXeK2W1CFoJWfw-1,1,1079.76,1.0,1079.76,1079.76,None,None,1079.76,1.0
4,o2-N-zzQyz1I7RlKZv2nBMqOoXw-2,1,2508.76,1.0,2508.76,2508.76,None,None,2508.76,1.0
5,o2-N-zzGvh4DCQfqhChcf-vXyQw-1,1,34.99,1.0,34.99,34.99,None,None,34.99,1.0
6,o2-N-zzDNsPtvRXmYFJVXM1UvDQ-1,1,1199.76,1.0,1199.76,1199.76,None,None,1199.76,1.0
7,o2-N-zzBwR7s9T6SMBhQ9gg2ntg-2,1,1704.76,1.0,1704.76,1704.76,None,None,1704.76,1.0
8,o2-N-zyk8f4BxRzWWJoXwUfb1SA-2,1,1548.76,1.0,1548.76,1548.76,None,None,1548.76,1.0
9,o2-N-zybLQf4eSAC2VEKT188oiA-1,1,1390.55,1.0,1390.55,1390.55,None,None,1390.55,1.0


In [ ]:
%%bigquery --project tefgfs 
SELECT
  -- Product SKU (dimension)
  productSKU AS Product_SKU,
  -- Product (dimension)
  v2ProductName AS Product,
  -- Product Variant (dimension)
  productVariant AS Product_Variant,
  -- Product Brand (dimension)
  productBrand AS Product_Brand,
  -- Product Category (Enhanced Ecommerce) (dimension)
  v2ProductCategory AS Product_Category_Enhanced_Ecommerce,
  -- Unique Purchases (metric)
  COUNT(hits.transaction.transactionId) AS Unique_Purchases,
  -- Quantity (metric)
  SUM(productQuantity) AS Quantity,
  -- Product Revenue (metric)
  SUM(productRevenue)/1000000 AS Product_Revenue,
  -- Avg. Price
  SUM(productRevenue)/1000000 / SUM(productQuantity) AS Avg_Price,
  -- Avg. QTY
  SUM(productQuantity) / COUNT(hits.transaction.transactionId) AS Avg_Quantity
FROM
  `tefgfs.103498486.ga_sessions_*`,
  UNNEST(hits) AS hits,
  UNNEST(product) AS product
WHERE
  _table_suffix BETWEEN '20200901'
  AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
  AND totals.visits = 1
  AND hits.eCommerceAction.action_type = '6'
GROUP BY
  1,
  2,
  3,
  4,
  5
ORDER BY
  6 DESC
LIMIT
  100

,Product_SKU,Product,Product_Variant,Product_Brand,Product_Category_Enhanced_Ecommerce,Unique_Purchases,Quantity,Product_Revenue,Avg_Price,Avg_Quantity
0,hw-only,hw-only,(not set),o2online,hw-only_smartphone,47432,47456,35394162.77,745.831144,1.000506
1,ecare Pack,ecare Pack,(not set),o2online,pack-only,8088,8088,1074877.20,132.897774,1.000000
2,hw-only,hw-only,(not set),o2online,hw-only_tablet,5783,5783,5334883.87,922.511477,1.000000
3,25213934,o2 Free S Boost Plus,vvl,o2online,postpaid_sim-only-vvl,4637,4637,3059307.12,659.760000,1.000000
4,36741761,bestandskunden-o2-free-starter-flex-online-par...,(not set),o2online,postpaid_sim-only,2833,2833,70796.67,24.990000,1.000000
...,...,...,...,...,...,...,...,...,...,...
95,34393088,o2 Free Unlimited Max,vvl,o2online,postpaid_bundle-vvl,187,187,408327.17,2183.567754,1.000000
96,25374673,privatkunden-o2-free-m-boost-online,(not set),o2online,postpaid_sim-only,186,186,156195.36,839.760000,1.000000
97,25644527,privatkunden-o2-free-m-online,(not set),o2online,postpaid_bundle,182,182,323922.74,1779.795275,1.000000
98,21341337,o2 Free M (2018),vvl,o2online,postpaid_sim-only-vvl,180,180,129556.80,719.760000,1.000000


---
# **TEF HitLevel-Feature von Gerald**


In [ ]:
%%bigquery --project tefgfs 

WITH date_range as (
SELECT * FROM `tefgfs.103498486.ga_sessions_*`
WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB(current_date(), INTERVAL 1 DAY)) 
)
SELECT   date,
         devicecategory,
         channelgrouping,
         customersegment,
         abtestvariant_cd112,
         distinct_products_viewed,
         distinct_products_purchased,
         
         SUM(prod_dsl_commercial)                                AS prod_dsl_commercial,
         SUM(prod_dsl_stayed)                                    AS prod_dsl_stayed,
         SUM(prod_o2_dsl_transactions)                           AS prod_o2_dsl_transactions,
         SUM(o2_prod_dsl_produkte)                               AS o2_prod_dsl_produkte,
         SUM(prod_dsl_basket)                                    AS prod_dsl_basket,
         SUM(page_dslfestnetz_prodverfuegbchk_sessions)          AS page_dslfestnetz_prodverfuegbchk_sessions,
         SUM(page_dslfestnetzorverfuegbarkchk_sessions)          AS page_dslfestnetzorverfuegbarkchk_sessions,
         SUM(page_dslfestnetz_verfuegbchk_ergebnis_sessions)     AS page_dslfestnetz_verfuegbchk_ergebnis_sessions,
         SUM(page_dslfestnetz_prodverfuegbchk_ergebnis_sessions) AS page_dslfestnetz_prodverfuegbchk_ergebnis_sessions,
         SUM(page_dslfestnetz_verfuegbchk_ergebnisall_sessions)  AS page_dslfestnetz_verfuegbchk_ergebnisall_sessions,
         SUM(page_warenkorb_sessions)                            AS page_warenkorb_sessions,
         SUM(page_checkout_mydata_sessions)                      AS page_checkout_mydata_sessions,
         SUM(page_checkout_myconnection_sessions)                AS page_checkout_myconnection_sessions,
         SUM(page_checkout_paymentdata_sessions)                 AS page_checkout_paymentdata_sessions,
         SUM(page_checkout_summary_sessions)                     AS page_checkout_summary_sessions,
         SUM(page_checkout_myappointment_sessions)               AS page_checkout_myappointment_sessions,
         SUM(page_warenkorb_prodverfuegbchk_sessions)            AS page_warenkorb_prodverfuegbchk_sessions,
         SUM(dsl_vcheck_down_sessions)                           AS dsl_vcheck_down_sessions,
         SUM(detail_views_count_by_action)                       AS detail_views_count_by_action,
         SUM(transaction_count_by_id)                            AS transaction_count_by_id
FROM    (
                  SELECT   -- Session level aggregated
                           -- filter dimensions for DS
                           date,
                           devicecategory,
                           channelgrouping,
                           CASE
                                                WHEN ucd175 LIKE "%Kunde%"
                                                OR     medium LIKE "%CRM%"
                                                OR     source LIKE "%CRM%"
                                                OR     ucd135 LIKE "%DSL- Ja%" THEN "Bestandskunde"
                                                ELSE "Neukunde"
                                         END AS customersegment,
                           ucd112 AS abtestvariant_cd112,
                           sessionid,
                           
                           -- counter columns for unique sessionIds per GA segment
                           COUNT(DISTINCT sessionid)                                                      AS prod_dsl_commercial,
                           COUNT(DISTINCT IF(bounces IS NULL, sessionid, NULL))                           AS prod_dsl_stayed,
                           COUNT(DISTINCT IF(eventcategory LIKE "%Summary Paynow%" 
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS prod_o2_dsl_transactions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/(dsl|internet)-festnetz/.*o2-my-(home|office).*|/e-shop/(dsl|internet)-festnetz/konfiguration/.*")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS o2_prod_dsl_produkte,                   
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "eshop.*basket|e-shop/warenkorb")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS prod_dsl_basket,
                                             
                           -- counter columns for unique sessionIds per step with page path regex
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "/e-shop/(dsl|internet)-festnetz/prod-verfuegbarkeitscheck/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_dslfestnetz_prodverfuegbchk_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/(dsl|internet)-festnetz/$|/e-shop/dsl-festnetz/prod-verfuegbarkeitscheck/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_dslfestnetzorverfuegbarkchk_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/(dsl|internet)-festnetz/verfuegbarkeitscheck-ergebnis/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_dslfestnetz_verfuegbchk_ergebnis_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/(dsl|internet)-festnetz/prod-verfuegbarkeitscheck-ergebnis/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_dslfestnetz_prodverfuegbchk_ergebnis_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/(dsl|internet)-festnetz(/prod-|/)verfuegbarkeitscheck-ergebnis/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_dslfestnetz_verfuegbchk_ergebnisall_sessions,
                                             
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/warenkorb/$")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_warenkorb_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/checkout/myData/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_checkout_mydata_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/checkout/myConnection/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_checkout_myconnection_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/checkout/paymentData/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_checkout_paymentdata_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/checkout/summary/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_checkout_summary_sessions,
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/checkout/myAppointment/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_checkout_myappointment_sessions,
                                             
                                             
                           -- basket directly from /e-shop/dsl-festnetz/prod-verfuegbarkeitscheck-ergebnis/
                           COUNT(DISTINCT IF(regexp_contains(pagepath, "^/e-shop/warenkorb/$")
                                             AND regexp_contains(previouspath, "^/e-shop/(dsl|internet)-festnetz/prod-verfuegbarkeitscheck-ergebnis/")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS page_warenkorb_prodverfuegbchk_sessions,
                                             
                           -- VCheck error DSLAvailabilityCheck down
                           COUNT(DISTINCT IF(eventcategory LIKE "%DSLAvailabilityCheck down%"
                                             AND regexp_contains(eventlabel, "not available|technischen")
                                             AND cd70 LIKE "%dsl%", sessionid, NULL))                     AS dsl_vcheck_down_sessions,
                                             
                           -- eec product data strings
                           STRING_AGG(DISTINCT v2productnamesdetailviews)                                 AS distinct_products_viewed,
                           STRING_AGG(DISTINCT v2productnamespurchases)                                   AS distinct_products_purchased,
                           COUNTIF(eecaction = "2")                                                       AS detail_views_count_by_action,
                           COUNT(DISTINCT transactionid)                                                  AS transaction_count_by_id
                  
                  FROM   (
                               SELECT   -- hit level aggregated
                                       date,
                                       concat(fullvisitorid, cast(visitstarttime AS string))                                             AS sessionid,
                                       device.devicecategory,
                                       channelgrouping,
                                       trafficsource.medium,
                                       trafficsource.source,
                                       totals.bounces,
                                       h.page.pagepath,
                                       lag(h.page.pagepath,1) OVER(partition BY fullvisitorid ORDER BY date, visitnumber, hitnumber ASC) AS previouspath,
                                       h.type,
                                       h.hitnumber,
                                       h.isentrance,
                                       totals.visits                                                                                     AS totalsvisits,
                                       h.eventinfo.eventcategory,
                                       h.eventinfo.eventlabel,
                                       -- product data strings per eec hit
                                       h.TRANSACTION.transactionid,
                                       h.ecommerceaction.action_type                                                                     AS eecaction,
                                       (
                                            SELECT   string_agg(DISTINCT v2productname ORDER BY v2productname)
                                            FROM     unnest(product)
                                            WHERE    NOT v2productname IS NULL
                                                     AND h.ecommerceaction.action_type = "2")                                        AS v2productnamesdetailviews,
                                       (
                                            SELECT   string_agg(DISTINCT v2productname ORDER BY v2productname)
                                            FROM     unnest(product)
                                            WHERE    NOT v2productname IS NULL
                                                     AND h.ecommerceaction.action_type = "6")                                        AS v2productnamespurchases,
                                       -- custom dimensions
                                       (
                                           SELECT ucd.value
                                           FROM   unnest(t.customdimensions) AS ucd
                                           WHERE  ucd.INDEX=112)                                                                      AS ucd112,
                                       (
                                           SELECT cd.value
                                           FROM   unnest(h.customdimensions) AS cd
                                           WHERE  cd.INDEX = 70)                                                                      AS cd70,
                                       (
                                           SELECT ucd.value
                                           FROM   unnest(t.customdimensions) AS ucd
                                           WHERE  ucd.INDEX = 135)                                                                    AS ucd135,
                                       (
                                           SELECT ucd.value
                                           FROM   unnest(t.customdimensions) AS ucd
                                           WHERE  ucd.INDEX = 175)                                                                    AS ucd175
                                FROM     date_range                     AS t,
                                         unnest(hits)                   AS h

                                WHERE   -- overall include conditions (shared across segments)
                                        totals.visits = 1
                                        AND regexp_contains(h.page.pagepath, "/(dsl|internet)-festnetz/")
                                        OR
                                        (
                                          SELECT cd.value
                                          FROM   unnest(h.customdimensions) AS cd
                                          WHERE  cd.INDEX = 70) LIKE "%dsl%")
                  WHERE
                    sessionid NOT IN (
                                SELECT concat(fullvisitorid, cast(visitstarttime AS string)) AS omitsessionid
                                FROM   date_range                                            AS t,
                                       unnest(hits)                                          AS h
                                WHERE  h.isentrance = true
                                       AND h.type = "PAGE"
                                       -- directBuyTraffic exclude
                                       AND regexp_contains(h.page.pagepath, "eshop.*confirmation|e-shop/bestellung|basket|warenkorb|cart|checkout|/check24de|/check24|/preis24|/mediamarkt|/saturn|reservierung"))
                  GROUP BY 1,
                           2,
                           3,
                           4,
                           5,
                           6 )
GROUP BY 1,
         2,
         3,
         4,
         5,
         6,
         7

,date,devicecategory,channelgrouping,customersegment,abtestvariant_cd112,distinct_products_viewed,distinct_products_purchased,prod_dsl_commercial,prod_dsl_stayed,prod_o2_dsl_transactions,o2_prod_dsl_produkte,prod_dsl_basket,page_dslfestnetz_prodverfuegbchk_sessions,page_dslfestnetzorverfuegbarkchk_sessions,page_dslfestnetz_verfuegbchk_ergebnis_sessions,page_dslfestnetz_prodverfuegbchk_ergebnis_sessions,page_dslfestnetz_verfuegbchk_ergebnisall_sessions,page_warenkorb_sessions,page_checkout_mydata_sessions,page_checkout_myconnection_sessions,page_checkout_paymentdata_sessions,page_checkout_summary_sessions,page_checkout_myappointment_sessions,page_warenkorb_prodverfuegbchk_sessions,dsl_vcheck_down_sessions,detail_views_count_by_action,transaction_count_by_id
0,20201012,mobile,SEA Mid Funnel,Neukunde,ab;DSL-PreOrder-Ja;DSL-PreOrder,"privatkunden-o2-my-home-l-flex,privatkunden-o2...",privatkunden-o2-my-home-l-flex,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,2,1
1,20201012,desktop,Direct,Bestandskunde,ab;DSL-PreOrder-Ja;DSL-PreOrder,None,None,36,33,0,3,0,1,14,0,0,0,0,0,1,0,0,0,0,0,0,0
2,20201012,desktop,Other_Referral,Neukunde,None,None,None,56,45,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20201012,mobile,Direct,Bestandskunde,None,None,None,1841,1507,0,7,0,0,332,0,0,0,0,1,0,0,0,0,0,0,0,0
4,20201012,desktop,Direct,Bestandskunde,None,None,None,705,683,0,8,0,1,92,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,20201012,desktop,SEA Mid Funnel,Neukunde,None,"privatkunden-o2-my-home-m-flex,privatkunden-o2...",None,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0
1413,20201012,desktop,SEA Lower Funnel,Neukunde,ab;DSL-PreOrder-Ja;DSL-PreOrder,privatkunden-o2-my-home-s,None,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
1414,20201012,desktop,CRM,Bestandskunde,MM_113_T320_Basic_Click&Collect;AB;a_Shop_Find...,None,None,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1415,20201012,mobile,Direct,Bestandskunde,None,"privatkunden-o2-my-home-l,privatkunden-o2-my-h...",None,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8,0


scheduled **mopGaSourcev1_freikarte**


In [ ]:
%%bigquery --project tefgfs 

WITH 
date_range as(
              Select * FROM `tefgfs.69128167.ga_sessions_*`
              WHERE _TABLE_SUFFIX = FORMAT_DATE("%Y%m%d", DATE_SUB(@run_date, INTERVAL 1 DAY))  #current_date()
),
-----------------
-- SESSION FIELDS
-----------------
session_view as(
              SELECT
                  date,
                  visitid,
                  clientid,
                  fullvisitorid,      
                  visitstarttime,
                  trafficSource.adwordsClickInfo.gclid AS GCLID,
                  device.deviceCategory AS sessionDeviceCategory,
                  trafficSource.adContent AS utmContent,
                  trafficSource.campaign AS utmCampaign,
                  trafficSource.isTrueDirect AS isTrueDirectSession,
                  trafficSource.keyword AS utmKeyword,
                  trafficSource.medium AS utmMedium,
                  trafficSource.source AS utmSource,
                  (SELECT STRING_AGG(DISTINCT cd.value) FROM UNNEST (hits) AS h, UNNEST(h.customdimensions) AS cd WHERE cd.index = 138) AS Customer_Group_URL_Hit,
                  (SELECT STRING_AGG(DISTINCT cd.value) FROM UNNEST(customdimensions) AS cd WHERE cd.index = 155) AS URLvoNumber_session,
                  (SELECT STRING_AGG(DISTINCT cd.value) FROM UNNEST (hits) AS h, UNNEST (h.customdimensions) AS cd WHERE cd.INDEX = 193) AS Application_Hit,
              FROM date_range
),
------------------------------------
-- SUBQUERY FOR PRODUCT DETAIL VIEWS
------------------------------------
detail_view as(
              SELECT
                date,
                visitid,
                clientID,
                fullvisitorid,
                visitstarttime,
                hp.v2ProductName AS Tarif_DetailView,
                count (hp.v2ProductName) as DetailView_cnt
              FROM date_range, UNNEST (hits) AS h, UNNEST (h.product) as hp
              WHERE h.eCommerceAction.action_type = "2" AND (hp.isImpression IS NULL)
              Group by 1,2,3,4,5,6
),
----------------------------
-- SUBQUERY FOR TRANSACTIONS
----------------------------
transaction_view as( -- SubQuery for Transaction Information
             SELECT
                date,
                visitid,
                clientID,
                fullvisitorid,
                visitstarttime,
                hp.v2ProductName AS Tarif_Transaction,
                h.transaction.transactionId AS gaTransactId,
                hp.productQuantity as Quantity,
            FROM date_range, UNNEST (hits) AS h, UNNEST (h.product) as hp
            WHERE h.eCommerceAction.action_type = "6" AND (hp.isImpression IS NULL)
),
----------------------------------------------------------------
-- COMBINATION OF SESSIONS WITH DETAIL VIEWS AND/OR TRANSACTIONS
----------------------------------------------------------------
detail_transaction as(
            SELECT
                a.*,
                Tarif_detailView,
                DetailView_cnt,
                "" AS Tarif_Transaction,
                "" AS gaTransactId,
                null as Quantity
            FROM session_view as a

          INNER JOIN(

            SELECT
                *
            FROM detail_view) as b ON a.date = b.date AND a.visitid = b.visitid AND a.clientid = b.clientid AND a.fullvisitorid = b.fullvisitorid AND a.visitstarttime = b.visitstarttime

          UNION ALL

            SELECT
                a.*,
                "" as Tarif_detailView,
                null as DetailView_cnt,
                Tarif_transaction,
                gaTransactId,
                Quantity
            FROM session_view as a
            
          INNER JOIN(
          
            SELECT
                *
            FROM transaction_view) as c ON a.date = c.date AND a.visitid = c.visitid AND a.clientid = c.clientid AND a.fullvisitorid = c.fullvisitorid AND a.visitstarttime = c.visitstarttime
),
----------------------------------------------
-- SESSIONS WITHOUT DETAIL VIEW OR TRANSACTION
----------------------------------------------
session_only as(
            SELECT
                s.*,
                "" as Tarif_detailView,
                null as DetailView_cnt,
                "" AS Tarif_Transaction,
                "" AS gaTransactId,
                null as Quantity
            FROM session_view as s

          LEFT JOIN
          
            (SELECT
            *
            FROM detail_transaction) as dt ON dt.date = s.date AND dt.visitid = s.visitid AND dt.clientId = s.clientId AND dt.fullvisitorid = s.fullvisitorid AND dt.visitstarttime = s.visitstarttime
            WHERE dt.date is null AND dt.visitid is null and dt.clientid is null and dt.fullvisitorid is null and dt.visitstarttime is null
)


-----------------------------------------------------------------------------------------------------------------
-- MAIN QUERY TO COMBINE SESSIONS WITH DETAILVIEWS, TRANSACTIONS AND REST OF SESSIONS WITHOUT PRODUCT INTERACTION
-----------------------------------------------------------------------------------------------------------------
SELECT
    *
FROM session_only

UNION ALL

SELECT
    *
FROM detail_transaction
ORDER BY visitid

DefaultCredentialsError: ignored

# **Impressions (campaigneData)**

In [ ]:
%%bigquery --project tefgfs 
SELECT _DATA_DATE, 	Advertiser_ID,Campaign_ID,	count(*) as impressions,  sum(Active_View_Viewable_Impressions) activeImps , FROM  (select *  from `tefgfs.CampaginManager_773418.impression_773418` LIMIT 100000)# where Active_View_Viewable_Impressions > 0 limit 100000 ) 
group by
_DATA_DATE,	 Advertiser_ID,Campaign_ID  order by _DATA_DATE	 #	Event_Time,Placement_ID,	Site_ID_DCM,City_ID,Ad_ID	,Advertiser_ID   

DefaultCredentialsError: ignored

In [ ]:
%%bigquery --project tefgfs 
SELECT * FROM `tefgfs.CampaginManager_773418.match_table_campaigns_773418` where Campaign Like "O2%" and Campaign_Start_Date like "202004%"  order by Campaign_ID	 desc   LIMIT 1000 

,Advertiser_ID,Campaign_ID,Campaign,Campaign_Start_Date,Campaign_End_Date,Billing_Invoice_Code,_LATEST_DATE,_DATA_DATE
0,8326841,24093163,O2_202000563-O2-Bestandskunden_2020-03-01_2020...,20200429,20201231,None,2020-10-12,2020-09-01
1,8326841,24093163,O2_202000563-O2-Bestandskunden_2020-03-01_2020...,20200429,20201231,None,2020-10-12,2020-09-30
2,8326841,24093163,O2_202000563-O2-Bestandskunden_2020-03-01_2020...,20200429,20201231,None,2020-10-12,2020-09-18
3,8326841,24093163,O2_202000563-O2-Bestandskunden_2020-03-01_2020...,20200429,20201231,None,2020-10-12,2020-09-19
4,8326841,24093163,O2_202000563-O2-Bestandskunden_2020-03-01_2020...,20200429,20201231,None,2020-10-12,2020-09-21
...,...,...,...,...,...,...,...,...
995,8326841,24002849,O2_202000496-Siegel-2020_2020-04-04_2020-04-10...,20200404,20200417,None,2020-10-12,2020-09-24
996,8326841,24002849,O2_202000496-Siegel-2020_2020-04-04_2020-04-10...,20200404,20200417,None,2020-10-12,2020-08-04
997,8326841,24002849,O2_202000496-Siegel-2020_2020-04-04_2020-04-10...,20200404,20200417,None,2020-10-12,2020-10-07
998,8326841,24002849,O2_202000496-Siegel-2020_2020-04-04_2020-04-10...,20200404,20200417,None,2020-10-12,2020-08-23


**Der Join zwischen MopGA und CampaingData klappt auf campaing- ebene**
https://developers.google.com/ads-data-hub/guides/cookie-matching

Select * from mopGA left join campaignData on campaignData.Campaign = mopGa.utmContent  

In [ ]:
%%bigquery --project tefgfs 

with mopGA as
(
Select  sessionDate,visitStartTime,fullVisitorId, channelGrouping,	campaignId,utmContent,	utmCampaign,	isTrueDirectSession	, utmKeyword, utmMedium	, utmSource
   from  `tefgfs.mmML.mopGaSourcev6_o2` where sessionDate like "202004%"   # isTrueDirectSession = 1
)
,campaignData as
 (  SELECT * FROM `tefgfs.CampaginManager_773418.match_table_campaigns_773418` where Campaign_Start_Date like "202004%" )  #Campaign Like "O2%" and 

Select* from mopGA  where utmSource like "O2%" order by utmSource  limit 1000
#Select distinct(campaignData.Campaign) from campaignData  where Campaign like "O2%" order by Campaign

# Select campaignData.Campaign , mopGa.utmContent	 from mopGA left join campaignData on campaignData.Campaign = mopGa.utmContent	  


   

,sessionDate,visitStartTime,fullVisitorId,channelGrouping,campaignId,utmContent,utmCampaign,isTrueDirectSession,utmKeyword,utmMedium,utmSource
0,20200401,1585751250,7147316363746897058,Affiliates,None,O2_AFF_CNT_EXA_15021,_dfa_773418:8326268:21563916,0,AFF_la_116012[__]_-GN0MUCOTYX,cpm,O2_AFF_CNT_EXA_15021
1,20200401,1585714305,2015543237795579899,Affiliates,None,O2_AFF_CNT_EXA_15021,_dfa_773418:8326268:21563916,1,AFF_la_116012[__]_-GN0MUCOTYX,cpm,O2_AFF_CNT_EXA_15021
2,20200401,1585751103,6095176116412736865,Affiliates,None,O2_AFF_CNT_EXA_15021,_dfa_773418:8326268:21563916,1,AFF_la_116012[__]_-GN0MUCOTYX,cpm,O2_AFF_CNT_EXA_15021
3,20200401,1585699303,5661937589823577402,Affiliates,None,O2_AFF_CNT_EXA_15021,(not set),1,AFF_la_116012[__]_-GN0MUCOTYX,cpm,O2_AFF_CNT_EXA_15021
4,20200401,1585736489,235701926978194038,Affiliates,None,O2_AFF_CNT_EXA_15021,(not set),1,AFF_la_116012[__]_-GN0MUCOTYX,cpm,O2_AFF_CNT_EXA_15021
...,...,...,...,...,...,...,...,...,...,...,...
995,20200401,1585744001,7483090056476125133,Affiliates,None,O2_AFF_CTA_EXA_15012,(not set),0,AFF_la_112060_-GN0MUCOTYX,cpm,O2_AFF_CTA_EXA_15012
996,20200401,1585737537,6465443709407489728,Affiliates,None,O2_AFF_CTA_EXA_15012,(not set),0,AFF_la_112060_-GN0MUCOTYX,cpm,O2_AFF_CTA_EXA_15012
997,20200401,1585770415,8382441678384197551,Affiliates,None,O2_AFF_CTA_EXA_15012,_dfa_773418:8326268:21563616,0,AFF_la_112566[__]_-GN0YCQFOTC,cpm,O2_AFF_CTA_EXA_15012
998,20200401,1585755623,1180937539208910754,Affiliates,None,O2_AFF_CTA_EXA_15012,_dfa_773418:8326268:21563616,1,AFF_ca_11094[__]_-GN0YCQFOTC,cpm,O2_AFF_CTA_EXA_15012


# die **mopGAv6** mit einigen Erweiterungen  #ew
https://github.com/Telefonica-Germany/TEFBigQuery/blob/master/mop-gaSource-v6-bl.sql

https://support.google.com/analytics/answer/3437719?hl=en

https://rpubs.com/Phippsy/bq-export


In [ ]:
%%bigquery --project tefgfs 

CREATE  OR REPLACE Table `tefgfs.mmML.mopGaSourcev6_o2` as 

WITH date_range as 
(
SELECT * FROM  `tefgfs.103498486.ga_sessions_20200401`
--WHERE _TABLE_SUFFIX = "20200209"--FORMAT_DATE("%Y%m%d", DATE_SUB(@run_date, INTERVAL 1 DAY)) 
)

SELECT
  s.date as sessionDate,
   #ew
   if( totals.visits=1 , CONCAT( fullvisitorid,"-",CAST(visitStartTime AS string)),"")   as visitTimeId ,
  s.visitId,
  s.clientId ,
  #ew
  fullVisitorId,
  visitStartTime,
    #ew
        device.operatingSystem as p2,  
        #trafficSource  as p3,
        device.browser as p9,
        device.deviceCategory as p3,
        device.mobileDeviceModel  as p4,
        device.mobileDeviceBranding  as p5,
         trafficSource.adwordsClickInfo.adNetworkType as p6,
        geoNetwork.city  as p7,
        geoNetwork.region as p8,
 
  trafficSource.adwordsClickInfo.gclid AS GCLID, 
  tr.gaTransactId ,
  transactionItemCats,
  transactionItemVariants,
  productCheckoutPromocode,
  quantity,
  device.deviceCategory AS sessionDeviceCategory,
  (SELECT MAX(h.time) FROM UNNEST(hits) as h) AS sessionDurationMS,
  (SELECT COUNT(h.time) FROM UNNEST(hits) as h WHERE h.type = "PAGE") AS sessionTotalPageviewCount,
  (SELECT COUNT(h.time) FROM UNNEST(hits) as h) AS sessionTotalInteractionCount,
  (SELECT COUNT(h.time) FROM UNNEST(hits) as h WHERE h.eCommerceAction.action_type = "2") AS sessionTotalProductViewCount,
  (SELECT COUNT(h.time) FROM UNNEST(hits) as h WHERE h.eCommerceAction.action_type = "3") AS sessionTotalAddToBasketCount,
  
------------------- GA Channel INFO     ----------------------------------
  channelGrouping,
  #ew
  if (channelGrouping like "%SEA%" or channelGrouping like "%Affiliates%" or channelGrouping like "%Display%" or  channelGrouping = "Other_SocialMedia",1,0) as isPaid,

  trafficSource.adContent as utmContent,
  trafficSource.campaign as utmCampaign,
  if (trafficSource.isTrueDirect,1,0) as isTrueDirectSession,
  trafficSource.keyword as utmKeyword,
  trafficSource.medium as utmMedium,
  trafficSource.source as utmSource,
#ew
trafficSource.adwordsClickInfo.adNetworkType as adNetworkType,
trafficSource.adwordsClickInfo.campaignId	as campaignId ,
trafficSource.adwordsClickInfo.creativeId	,
trafficSource.adwordsClickInfo.criteriaId	,
trafficSource.adwordsClickInfo.criteriaParameters,

  
------------------- TRBO SEGMENTS       ----------------------------------
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_strongestAffinityManufacturer_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 10) as trboSegment_strongestAffinityManufacturer_User,
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_strongestAffinityClientType_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 11) as trboSegment_strongestAffinityClientType_User,
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_strongestAffinityGeneralTariff_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 12) as trboSegment_strongestAffinityGeneralTariff_User,
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_strongestAffinityOfferType_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 13) as trboSegment_strongestAffinityOfferType_User,
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_latestAffinityManufacturer_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 14) as trboSegment_latestAffinityManufacturer_User,
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_latestAffinityClientType_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 15) as trboSegment_latestAffinityClientType_User,
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_latestAffinityGeneralTariff_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 16) as trboSegment_latestAffinityGeneralTariff_User,
  (SELECT
     STRING_AGG(DISTINCT hcd.value) as trboSegment_latestAffinityOfferType_User
   FROM UNNEST(hits) as h, UNNEST(customdimensions) as hcd
   WHERE hcd.index = 17) as trboSegment_latestAffinityOfferType_User,
   
------------------- INTEREST INDICATORS ----------------------------------
  (SELECT hwName FROM(SELECT cd.value as hwName, SUM(
    CASE ecommerceaction.action_type
      WHEN "0" THEN 1  --product list impression
      WHEN "2" THEN 10 --product detail view
      WHEN "3" THEN 40 --product add to basket
      WHEN "5" THEN 20 --product checkout
      ELSE 0
    END) as interestIndicatorRating FROM UNNEST(hits) as h, UNNEST(product) as p, UNNEST(p.customdimensions) as cd WHERE cd.index=168 GROUP BY hwName ORDER BY interestIndicatorRating DESC) LIMIT 1) as highInterest_hardwareName,
  (SELECT interestIndicatorRating FROM(SELECT cd.value as hwName, SUM(
    CASE ecommerceaction.action_type
      WHEN "0" THEN 1  --product list impression
      WHEN "2" THEN 10 --product detail view
      WHEN "3" THEN 40 --product add to basket
      WHEN "5" THEN 20 --product checkout
      ELSE 0
    END) as interestIndicatorRating FROM UNNEST(hits) as h, UNNEST(product) as p, UNNEST(p.customdimensions) as cd WHERE cd.index=168 GROUP BY hwName ORDER BY interestIndicatorRating DESC) LIMIT 1) as highInterest_hardwareName_score,
  (SELECT pName FROM(SELECT p.v2ProductName as pName, SUM(
    CASE ecommerceaction.action_type
      WHEN "0" THEN 1  --product list impression
      WHEN "2" THEN 10 --product detail view
      WHEN "3" THEN 40 --product add to basket
      WHEN "5" THEN 20 --product checkout
      ELSE 0
    END) as interestIndicatorRating FROM UNNEST(hits) as h, UNNEST(product) as p GROUP BY pName ORDER BY interestIndicatorRating DESC) LIMIT 1) as highInterest_productName,
  (SELECT interestIndicatorRating FROM(SELECT p.v2ProductName as pName, SUM(
    CASE ecommerceaction.action_type
      WHEN "0" THEN 1  --product list impression
      WHEN "2" THEN 10 --product detail view
      WHEN "3" THEN 40 --product add to basket
      WHEN "5" THEN 20 --product checkout
      ELSE 0
    END) as interestIndicatorRating FROM UNNEST(hits) as h, UNNEST(product) as p GROUP BY pName ORDER BY interestIndicatorRating DESC) LIMIT 1) as highInterest_productName_score,
------------------- CHURN SEGMENT       ----------------------------------
    (SELECT MAX(
      IF(h.page.pagepath LIKE "/postpaid/ecare/kuendigungvormerken/success%", 3,
      IF(REGEXP_CONTAINS(h.page.searchKeyword, "(kündig|kuendig|kundig)"), 2,
      IF(h.page.pagepath LIKE "/service/kuendigung%", 1, 0)))) as gaChurnLevel
     FROM UNNEST(hits) as h) as gaChurnLevel,
    
    
------------------- GA SEGMENTS         ----------------------------------
  IF((SELECT
     MAX(IF(REGEXP_CONTAINS(v2ProductCategory, "postpaid|hw-only|smartphone"), 1, 0)) as isSessionCommProduct
   FROM UNNEST(hits) as h, UNNEST(product) as p) = 1, 1, 0) as isSessionCommProduct, --IF 1=1 for null value handling
   IF((SELECT
     MAX(IF((cd.value = "postpaid" AND cd.index = 70), 1, 0)) as isSessionCd70Postpaid 
   FROM UNNEST(hits) as h, UNNEST(h.customdimensions) as cd) = 1, 1, 0) as isSessionCd70Postpaid,
  (SELECT 
     STRING_AGG(page.pagePath) as landingpage
   FROM UNNEST(hits) as h WHERE h.type = "PAGE" AND h.isEntrance = TRUE) as landingpage,
  IF(totals.bounces IS NULL, 0, 1) as isBounce,
  (SELECT 
     MAX(IF(REGEXP_CONTAINS(page.pagePath, r"^/mein-o2/.*|^/service/.*|^/kundencenter/.*|^/priority/.*") OR 
        page.hostname IN ("freundewerben.o2online.de", "login.o2online.de", "edel.o2online.de") OR
        (hcd.index = 193 AND hcd.value in ("ecare","ecare-postpaid","ecare-prepaid","welcome","blog","hilfe","freischalten","b2b","econtract","mehr-o2","root-priority")), 1, 0)) as isServiceTraffic
   FROM UNNEST(hits) as h, UNNEST(h.customdimensions) as hcd) as isServiceTraffic,
  (SELECT 
     MAX(IF(page.pagePath LIKE "%/dsl-festnetz/%" OR (hcd.index = 70 AND hcd.value LIKE "'%dsl%"), 1, 0 )) as isDSLSession
   FROM UNNEST(hits) as h, UNNEST(h.customdimensions) as hcd) as isDSLSession,
  (SELECT 
     SIGN(COUNT(1))
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd WHERE hcd.index = 193 AND hcd.value LIKE "%pixie%" LIMIT 1) as isPixieSession,
  (SELECT 
     CAST(STRING_AGG(DISTINCT hcd.value) = 'pixie' as INT64)
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd WHERE hcd.index = 193) as isPixieOnlySession,
  IF((SELECT 
       IF(isEntrance = true AND REGEXP_CONTAINS(page.pagePath, r"eshop.*confirmation|e-shop/bestellung|basket|warenkorb|cart|checkout|/check24de|/check24|/preis24|/mediamarkt|/saturn|reservierung"), 1, 0 ) as isDirectBuySession
     FROM UNNEST(hits) WHERE type="PAGE" LIMIT 1) = 1,1,0) as isDirectBuySession,
(SELECT
     MAX(IF(hcd.value = "econtract",1,0)) as isVVLSession
   FROM UNNEST(hits), UNNEST(customdimensions) as hcd WHERE hcd.index = 193) as isVVLSession,
  IF((SELECT MAX(
  IF((sessionCD.index = 175
  AND sessionCD.value LIKE '%Kunde%') OR
  trafficSource.medium LIKE '%CRM%' OR
  trafficSource.source LIKE '%CRM%', 1, 0)) FROM UNNEST(s.customDimensions) AS sessionCD) = 1,1,0) as isCustomerSession,
  (SELECT
     MAX(IF(REGEXP_CONTAINS(page.pagePath, r"eshop.*basket|e-shop/warenkorb|e-shop/checkout|e-shop/bestellung|e-store/cart"), 1, 0)) as isCartAdd
   FROM UNNEST(hits)) as isCartAdd,
  (SELECT
     MAX(IF(REGEXP_CONTAINS(page.pagePath, r"eshop.*confirmation|/pixie/confirmation/|/e-shop/bestellung/") OR
        eventInfo.eventCategory = "Summary Paynow", 1, 0)) as isEShopTransaction
   FROM UNNEST(hits)) as isEShopTransaction,
   
   IF((SELECT
     MAX(IF(REGEXP_CONTAINS(p.v2ProductCategory, r"postpaid_bundle$|postpaid_sim-only$"), 1, 0)) as ProductCategoryFilter_PO
    FROM UNNEST (hits) as h, UNNEST (h.product) as p WHERE h.transaction.transactionId IS NOT NULL)=1,1,0) as ProductCategoryFilter_PO,
    

   IF(REGEXP_CONTAINS(
              (SELECT 
                    STRING_AGG(page.pagePath) as landingpage
               FROM UNNEST(hits) as h WHERE h.type = "PAGE" AND h.isEntrance = TRUE), 
       r"^/e-shop/o2shop-partnering/.*|^/e-shop/edel.*|^/business/.*|^/e-shop/o2shop-partnering/.*"), 1, 0) as LandingpageFilter_PO,
   IF((REGEXP_CONTAINS(trafficSource.source, r"^letter\.sky\.de.*|^vertriebspartner\.de\.o2\.com.*")), 1, 0) as SourceFilter_PO,
   IF((REGEXP_CONTAINS(channelgrouping,  r"(?i:Allother)")), 1, 0) as ChannelgroupingFilter_PO,
   
FROM
  date_range AS s
  LEFT JOIN (
    SELECT 
      clientId, 
      visitId, 
      h.transaction.transactionId gaTransactId,
       
      STRING_AGG(DISTINCT p.v2ProductCategory) as transactionItemCats, 
      STRING_AGG(DISTINCT p.productVariant) as transactionItemVariants,
      STRING_AGG(DISTINCT (SELECT pCd.value as productCheckoutPromocode FROM UNNEST(p.customDimensions) as pCd WHERE pCd.index = 62)) as productCheckoutPromocode,
      p.productQuantity as quantity,       
    FROM date_range s, UNNEST(hits) as h, UNNEST(h.product) as p-- , UNNEST(p.customDimensions) as pCd
    WHERE h.transaction.transactionId IS NOT NULL --AND (pCd.index = 62 OR NOT EXISTS (SELECT index FROM p.customDimensions WHERE index = 62))
    GROUP BY clientId, visitId, gaTransactId, p.productQuantity
  ) AS tr ON tr.visitId = s.visitId AND tr.clientId = s.clientId

""


In [ ]:
 %%bigquery --project tefgfs 
 
 Select * From `tefgfs.mmML.mopGaSourcev6_o2`  where isPaid = 1 and gaTransactId <> "None"

,sessionDate,visitTimeId,visitId,clientId,fullVisitorId,visitStartTime,p2,p9,p3,p4,p5,p6,p7,p8,GCLID,gaTransactId,transactionItemCats,transactionItemVariants,productCheckoutPromocode,quantity,sessionDeviceCategory,sessionDurationMS,sessionTotalPageviewCount,sessionTotalInteractionCount,sessionTotalProductViewCount,sessionTotalAddToBasketCount,channelGrouping,isPaid,utmContent,utmCampaign,isTrueDirectSession,utmKeyword,utmMedium,utmSource,adNetworkType,campaignId,creativeId,criteriaId,criteriaParameters,trboSegment_strongestAffinityManufacturer_User,trboSegment_strongestAffinityClientType_User,trboSegment_strongestAffinityGeneralTariff_User,trboSegment_strongestAffinityOfferType_User,trboSegment_latestAffinityManufacturer_User,trboSegment_latestAffinityClientType_User,trboSegment_latestAffinityGeneralTariff_User,trboSegment_latestAffinityOfferType_User,highInterest_hardwareName,highInterest_hardwareName_score,highInterest_productName,highInterest_productName_score,gaChurnLevel,isSessionCommProduct,isSessionCd70Postpaid,landingpage,isBounce,isServiceTraffic,isDSLSession,isPixieSession,isPixieOnlySession,isDirectBuySession,isVVLSession,isCustomerSession,isCartAdd,isEShopTransaction,ProductCategoryFilter_PO,LandingpageFilter_PO,SourceFilter_PO,ChannelgroupingFilter_PO
0,20200401,929913408180961857-1585734793,1585734793,216512337.1585431105,929913408180961857,1585734793,Android,Chrome,mobile,SM-A307FN,Samsung,Google Search,Eggenfelden,Bavaria,CjwKCAjw95D0BRBFEiwAcO1KDMrob_ow-EccFIinRxHukT...,o2-N-SW3pRUdLS-OEP6Ok6rbb0w-1,smartphone,(not set),NO_PROMOTION,1,mobile,348369,11,149,1,1,SEA Lower Funnel,1,o2 Handy- & DSL-Tarife,SB_Brand,0,o2,cpc,google,Google Search,42805603.0,4.279682e+11,10590282.0,o2,samsung,NK,n/a,HW-Only,samsung,NK,n/a,HW-Only,privatkunden-samsung-galaxy-note-10-lite-schwa...,190,hw-only,230,0,1,1,/,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,20200401,4952470397639333988-1585757898,1585757898,1153086870.1542330468,4952470397639333988,1585757898,Android,Chrome,mobile,SM-G930U,Samsung,Google Search,Berlin,Berlin,CjwKCAjw95D0BRBFEiwAcO1KDHxzCizVIKg2Qj0Al6HvLt...,o2-N-i5xHRS2uQ-aOXOKfvlcrnw-2,bundle,(not set),YOU2020363,1,mobile,985244,13,209,3,0,SEA Lower Funnel,1,o2 Handy- & DSL-Tarife,SB_Brand,0,o2,cpc,google,Google Search,42805603.0,4.279682e+11,10590282.0,o2,samsung,NK,postpaid,Bundle,samsung,NK,postpaid,Bundle,privatkunden-samsung-galaxy-s20-plus-4g-schwar...,80,privatkunden-o2-free-m-flex-you-flex-36m-11,80,0,0,1,/,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,20200401,5033950402608374148-1585742851,1585742851,1172057912.1550328196,5033950402608374148,1585742851,Macintosh,Firefox,desktop,None,None,Google Search,Hamburg,Hamburg,EAIaIQobChMI6NfTuJjH6AIVSrDtCh2l2QCDEAAYASAAEg...,o2-B-zeI6U94rRIaNYwnWZx5N6g-2,bundle,(not set),NO_PROMOTION,1,desktop,805221,12,168,10,1,SEA Lower Funnel,1,(not set),SB_Brand_Young_People,0,o2,cpc,google,Google Search,154449913.0,3.809083e+11,10590282.0,o2,apple,BK,prepaid,Bundle,apple,BK,prepaid,Bundle,privatkunden-apple-iphone-11-pro-64gb-nachtgru...,250,privatkunden-o2-free-unlimited-max-online-hwv-...,210,0,0,1,/,0,0,0,0,0,0,0,1,1,1,0,0,0,0
3,20200401,9030940169089912462-1585762121,1585762121,2102679612.1583943310,9030940169089912462,1585762121,Android,Chrome,mobile,SM-G975F,Samsung,Google Search,Stuttgart,Baden-Wurttemberg,CjwKCAjw95D0BRBFEiwAcO1KDLBddzazus17L4OQH6WFrd...,o2-C-7kIt4wk3QEegOwPHTPNYmg-2,bundle,(not set),NO_PROMOTION,1,mobile,460400,9,157,2,1,SEA Lower Funnel,1,(not set),SB_Brand_Young_People,0,o2,cpc,google,Google Search,154449913.0,4.291288e+11,10590282.0,o2,samsung,NK,postpaid,Bundle,samsung,NK,postpaid,Bundle,studenten-samsung-galaxy-s20-ultra-5g-128gb-gr...,200,studenten-o2-free-m-boost-online-junge-leute-h...,150,0,1,1,/,0,0,0,0,0,0,0,0,1,1,0,0,0,0
4,20200401,1914641446561139668-1585728904,1585728904,445787200.1585728468,1914641446561139668,1585728904,Android,Chrome,mobile,CLT-L29,Huawei,Google Search,Bruchsal,Baden-Wurttemberg,CjwKCAjw95D0BRBFEiwAcO1KDPm2HVXV4tKnFAJptaIlKr...,o2-N-baugsr_FTjW

In [ ]:
 %%bigquery --project tefgfs 
 Select 
 (Select count(distinct(gaTransactId)) From `tefgfs.mmML.mopGaSourcev6_o2`  where isPaid = 1 and gaTransactId <> "None") as paidTransactions ,
 (Select count(distinct(gaTransactId))  From `tefgfs.mmML.mopGaSourcev6_o2`  where isPaid = 0 and gaTransactId <> "None") as freeTransactions,
From `tefgfs.mmML.mopGaSourcev6_o2` limit 1
 

,paidTransactions,freeTransactions
0,1645,3225


**xxx**

In [ ]:
%%bigquery --project tefgfs 
Select  sessionDate,visitStartTime,fullVisitorId, channelGrouping,	campaignId,utmContent,	utmCampaign,	isTrueDirectSession	, utmKeyword, utmMedium	, utmSource
   from  `tefgfs.mmML.mopGaSourcev6_o2` where  isTrueDirectSession = 1  Limit 10000

,sessionDate,visitStartTime,fullVisitorId,channelGrouping,campaignId,utmContent,utmCampaign,isTrueDirectSession,utmKeyword,utmMedium,utmSource
0,20200401,1585760418,3690195091525151501,AllOther_undefined_internal_partnering_eRetail,None,online,(not set),1,service,nf,online
1,20200401,1585765482,1408632189523202541,Direct,None,O2_DTI_APP_TEF_11022,vvl,1,smartbanner,app,O2_DTI_APP_TEF_11022
2,20200401,1585732848,7934603789141612998,Direct,None,O2_DTI_APP_TEF_11034,cloud,1,angebote,app,O2_DTI_APP_TEF_11034
3,20200401,1585731930,6039758892099734735,Direct,None,O2_DTI_APP_TEF_11077,galaxy-s20-serie,1,angebote,app,O2_DTI_APP_TEF_11077
4,20200401,1585758663,1314523456043713410,Direct,None,O2_DTI_APP_TEF_11074,amazon-music-unlimited,1,angebote,app,O2_DTI_APP_TEF_11074
...,...,...,...,...,...,...,...,...,...,...,...
9995,20200401,1585714345,8927077431638384412,Direct,None,O2_DTI_APP_TEF_11115,corona-deals,1,angebote,app,O2_DTI_APP_TEF_11115
9996,20200401,1585736928,8382058262346697805,Direct,None,O2_DTI_APP_TEF_11115,we-stay-connected,1,angebote,app,O2_DTI_APP_TEF_11115
9997,20200401,1585699318,7692800519108963922,Direct,None,O2_DTI_APP_TEF_11115,we-stay-connected,1,angebote,app,O2_DTI_APP_TEF_11115
9998,20200401,1585707168,1382082909339583892,Direct,None,O2_DTI_APP_TEF_11115,we-stay-connected,1,angebote,app,O2_DTI_APP_TEF_11115


In [ ]:
%%bigquery --project tefgfs 

CREATE  OR REPLACE Table `tefgfs.mmML.KW_analyseData` as

Select *  EXCEPT( rowId,  productCheckoutPromocode) 
from (Select if(gaTransactId is null,0,1) as label, *  FROM   `tefgfs.ReportingData.mopGaSourcev6_o2_*`  WHERE  _TABLE_SUFFIX BETWEEN "20200901" AND "20200930" ) 
where   sessionTotalInteractionCount > 4 and sessionDurationMS > 500  and isBounce	= 0

In [ ]:
%%bigquery --project tefgfs 
Select * from `tefgfs.mmML.KW_analyseData`  group by GCLID order by sessionDate limit 200

# KW-DataStudioReport

# Studie

**Design**
Der überwiegende Teil der User kommt öfters auf unsere WebSite
Ausgangslage:die von mir erweiterte mopGAv6 - incls Activierungen, isPaid (für paid Traffic)  und  **preConversionVisits**:  Anzahl der SideBesuche VOR Transaction.  Biete preConversionVisits als KW xgboost Indikator an. 

**InputDaten:   mopGAv6pp - isPaid** - **has converged.**
Von den Besuchern die offSite Werbung von uns gesehen haben (pageImpressions) aber NICHT mit einme Klick reagiert haben der sie zu unser Side geführt hat - wissen wir nichts !!!!!  ( google gibt nichts weiter)
Wurd der User aktiv (auf Banner geklickt - oder in adwords gesucht (sea oder seo)) wissen wir das (customerChannel).  
Wir interessieren uns jetzt nur für die paid channel !!!! 
Natürlich wird upper channel display - non klick Werbung (Lidwardsäulen) auch helfen Interesse zu wecken - wir wissen dass diese eine Sättigungsfunktion ist - (also nicht beliebig mit Geld gesteigert werden kann)     rx(1-x)  .. aber r (in Form von Havas umstrittener Grenzkostenfunktion) ist nicht wirklich bekannt.   
Wir konzentrieren uns in dieser Studie daher ganz auf Wiederkehrer deren zweiter Kontakt via isPaid-Traffic (Traffic der zu uns geführt hat und Geld gekostet hat und (für SEA)  kommt und  auch mit einer aktuelle Kampagne in Verbindung gebracht wird.
Das entspricht immer noch x% unserer gesamten GrossAdds.

Als firstKontaktler haben uns als k1% Besucher   besucht.  Von denen haben y1% gleich convergiert - und von diesen sind z1 über isPaid gekommen.
Der absolute überwiegende Teil der Käufer kam als Mehrfachbesucher --
und von denen gab es lediglich r% die ausschließlich über nonPaid kamen.

Heißt:  der überwiegende Anteil unseres Geschäfts läuft über Mehrfachbsuche - von denen der überwiegende Anteil marketingSpendings kostet

In diesere Studie (Zeitraum 6 Monate)  untersuchen wir nun diese isPaid  - Wiederkehrer die am Ende kaufen, und zwar immer genau den Verlauf bis zur letzten Conversion.
Für jeden Besuch berechnen wir die Kaufwahrscheinlichkeit KW.
(ist am Ende, beim Kauf 1)
Wir wissen von Ihnen dass sie Werbekontakte mit uns hatten (isPaid)
Die Frage ist nun: 
a Welchen Verlauf hat ihre KW während der Besuche ?
    (sie sollte zunehmen ) ? 
b Betrachten wir mal die double-Visitors  (die also beim zweiten Besuch kaufen): Wie ist die KW-Verteilung aus - das  KW-Histogramm:  Gibt es einen/mehrere HaufüngsPeaks -  ein High bei niedrigen KW-Werten oder bei den Low-KW-Werten   - oder ist das eher gleichverteilt - spielt die KW also keine Rolle für den nachfolgenden Kauf.

**statische Tests**: ... 

Angenommen es gibt ein High bei KWLow.   Dann wissen wir dass es Sinn macht unsere Werbemittel auf KWLow-Teilnehmer zu konzentrieren.
Dh. der cpo sinkt und wir werden den gleichen Orders-Hub mit geringern Spendings erreichen die wir dann für andere Kampagnen einsetzen können.